# 2.6 Application Programming Interface (API)

### 2.6.5.4 Using APIs without Authentication

Send a GET request and validate the response:

```python
# Importing the library
import requests

# Sending a GET request
response = requests.get('https://api.example.com/data')

if response.status_code == 200:
    data = response.json()
else:
    print(f'Failed to retrieve data: {response.status_code}')
```

Handle errors in API requests:

```python
try:
    response = requests.get('https://api.example.com/data')
    response.raise_for_status()
    data = response.json()
except requests.exceptions.HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'An error occurred: {err}')
```

Use sessions for multiple requests:

```python
import requests

with requests.Session() as session:
    try:
        response = session.get('https://api.example.com/data')
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'An error occurred: {err}')
```

### 2.6.5.5 Concurrent API Calls with Multithreading

Make concurrent API calls using ThreadPoolExecutor:

```python
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_url(url):
    with requests.Session() as session:
        return session.get(url)

urls = ['https://api.example.com/data1', 'https://api.example.com/data2']

with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(fetch_url, url): url for url in urls}
    for future in as_completed(future_to_url):
        url = future_to_url[future]
        try:
            response = future.result()
            data = response.json()
        except Exception as exc:
            print(f'{url} generated an exception: {exc}')
        else:
            print(f'The page {url} is {len(data)} bytes')
```

### 2.6.5.7 API Requests with Authentication

Bearer Token Authentication:

```python
import requests

# Your Bearer token (typically obtained after an authorization process)
bearer_token = 'your_bearer_token'

# Prepare HTTP headers with the authentication token
headers = {
    'Authorization': f'Bearer {bearer_token}'
}

# Send a GET request with authentication
response = requests.get('https://api.example.com/protected', headers=headers)

# Validate the response
if response.ok:
    print('Response received:', response.json())
else:
    print('Error received:', response.status_code)
```

Basic HTTP Authentication:

```python
from requests.auth import HTTPBasicAuth
import requests

response = requests.get(
    'https://api.example.com/basic-auth',
    auth=HTTPBasicAuth('username', 'password')
)

# Ensure the response is valid
if response.ok:
    data = response.json()
else:
    print('Failed to retrieve data:', response.status_code)
```

Proxy Authentication:

```python
import os

os.environ['HTTP_PROXY']='http://monproxy.fr:1234'
os.environ['HTTPS_PROXY']= 'https://monproxy.fr:1234'

# Creating the dictionary containing proxy URLs
PROXIES = {
'http': os.environ['HTTP_PROXY'],
'https': os.environ['HTTPS_PROXY'] }

r = requests.get('http://www.url.com', proxies=PROXIES)
```

Cookie Authentication:

```python
import requests

# Create a session to maintain cookies
session = requests.Session()

# Log in to obtain the session cookie
session.post('https://api.example.com/login', 
             data={'username': 'name', 
                   'password': 'password'})

# The session cookie is now stored in the session
response = session.get('https://api.example.com/protected')

# Check the response
if response.ok:
    print('Response received from the protected API.')
else:
    print('Request error:', response.status_code)
```

## 2.6.6 Practical Illustration

Example of using the French Address API:

```python
import json
import requests

# The address we have is the following: 4 bd du port
# We have the postcode 43330 and it seems valid, let's launch
# a first test to verify if we can extract a valid address
url = "https://api-adresse.data.gouv.fr/search/?q=8+bd+du+port&postcode=44380"

with requests.Session() as session:
    try:
        response = session.get(url)
        response.raise_for_status()

        # Selecting only the first item from the file
        data = response.json()['features'][0]

        # Using the dumps method from the json library for
        # a more aesthetically pleasing output with print
        print(json.dumps(data, indent=2))
        
    except requests.exceptions.HTTPError as http_err:
        print(f'An HTTP error occurred: {http_err}')

    except Exception as err:
        print(f'An error occurred: {err}')
```